In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from string import punctuation
from sklearn.feature_extraction import stop_words
stopwords = stop_words.ENGLISH_STOP_WORDS

In [90]:
df_raw = pd.read_csv('../data/mbti_1.csv')

In [91]:
df_test = df_raw[:650]
df_test.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [92]:
#  Strip quotes
def remove_first_last_quote(dataframe, col):
    dataframe[col] = dataframe[col].str[1:-1]
    return dataframe
df_raw_no_quote = remove_first_last_quote(df_test, 'posts')
df_raw_no_quote.head()

/home/asa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,type,posts
0,INFJ,http://www.youtube.com/watch?v=qsXHcwe3krw|||h...
1,ENTP,I'm finding the lack of me in these posts very...
2,INTP,Good one _____ https://www.youtube.com/watc...
3,INTJ,"Dear INTP, I enjoyed our conversation the ot..."
4,ENTJ,You're fired.|||That's another silly misconcep...


In [93]:
df_raw_no_quote.posts = df_raw_no_quote.posts.str.split('\|\|\|')
df_raw_no_quote.head()

,type,posts
0,INFJ,"[http://www.youtube.com/watch?v=qsXHcwe3krw, h..."
1,ENTP,[I'm finding the lack of me in these posts ver...
2,INTP,[Good one _____ https://www.youtube.com/wat...
3,INTJ,"[Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,"[You're fired., That's another silly misconcep..."


In [94]:
# df_raw_no_quote['posts_split'] = ""

for user in range(df_raw_no_quote.shape[0]):
#     breakpoint()
    user_word_list = []
    for post_lst in range(len(df_raw_no_quote['posts'][user])):
        user_word_list += df_raw_no_quote['posts'][user][post_lst].split()
    user_word_list2 = [x for x in user_word_list if not x.startswith("http")]
    df_raw_no_quote['posts'][user] = user_word_list2
    
df_raw_no_quote.head()

,type,posts
0,INFJ,"[enfp, and, intj, moments, sportscenter, not, ..."
1,ENTP,"[I'm, finding, the, lack, of, me, in, these, p..."
2,INTP,"[Good, one, _____, Of, course,, to, which, I, ..."
3,INTJ,"[Dear, INTP,, I, enjoyed, our, conversation, t..."
4,ENTJ,"[You're, fired., That's, another, silly, misco..."


In [95]:
len(df_raw_no_quote['posts'][2])

880

In [96]:
df_word_list_by_type_raw = df_raw_no_quote.groupby('type').agg({'posts': 'sum'})
df_word_list_by_type_raw.head(20)

,posts
type,
ENFJ,"[51, :o, I, went, through, a, break, up, some,..."
ENFP,"[He, doesn't, want, to, go, on, the, trip, wit..."
ENTJ,"[You're, fired., That's, another, silly, misco..."
ENTP,"[I'm, finding, the, lack, of, me, in, these, p..."
ESFJ,"[Why, not?, Any, other, ESFJs, originally, mis..."
ESFP,"[Edit:, I, forgot, what, board, this, was, on...."
ESTJ,"[his, is, such, a, catch, 22, I'm, here!, Alth..."
ESTP,"[plinter, Cell, Blacklist, for, Xbox, 360., ES..."
INFJ,"[enfp, and, intj, moments, sportscenter, not, ..."


In [97]:
len(df_word_list_by_type_raw['posts'][11])

124002

In [50]:
type_list = df_raw['type'].head(650)
print(set(type_list))

{'INFJ', 'ISTJ', 'ESTP', 'ESTJ', 'INTJ', 'ENTP', 'ISTP', 'ENFP', 'ESFP', 'ESFJ', 'ISFP', 'INTP', 'ISFJ', 'ENTJ', 'INFP', 'ENFJ'}


In [7]:
def split_df_col_text_by_delim(df, col, delim=" "):
    df[col] = df[col].str.split(delim)
    return df

In [8]:
df_raw_no_quote.shape[0]

650

In [ ]:
for user in range(df_raw_no_quote.shape[0]):
    user_word_list = []
    for post in range(len(df_raw_no_quote['posts'][user])):
        len(df_raw_no_quote['posts'][user][post].str.split(" ")
        user_word_list += df_raw_no_quote['posts'][user][post]

In [33]:
df_raw_no_quote['posts'][0]
user = 0
post_lst = 2
df_raw_no_quote['posts'][user][post_lst].split()

['enfp',
 'and',
 'intj',
 'moments',
 'https://www.youtube.com/watch?v=iz7lE1g4XM4',
 'sportscenter',
 'not',
 'top',
 'ten',
 'plays',
 'https://www.youtube.com/watch?v=uCdfze1etec',
 'pranks']

In [58]:
len(df_raw_no_quote['posts_split'][0])

581

In [59]:
df_raw_no_quote['posts_split'][0]

['enfp',
 'and',
 'intj',
 'moments',
 'sportscenter',
 'not',
 'top',
 'ten',
 'plays',
 'pranks',
 'What',
 'has',
 'been',
 'the',
 'most',
 'life-changing',
 'experience',
 'in',
 'your',
 'life?',
 'On',
 'repeat',
 'for',
 'most',
 'of',
 'today.',
 'May',
 'the',
 'PerC',
 'Experience',
 'immerse',
 'you.',
 'The',
 'last',
 'thing',
 'my',
 'INFJ',
 'friend',
 'posted',
 'on',
 'his',
 'facebook',
 'before',
 'committing',
 'suicide',
 'the',
 'next',
 'day.',
 'Rest',
 'in',
 'peace~',
 'Hello',
 'ENFJ7.',
 'Sorry',
 'to',
 'hear',
 'of',
 'your',
 'distress.',
 "It's",
 'only',
 'natural',
 'for',
 'a',
 'relationship',
 'to',
 'not',
 'be',
 'perfection',
 'all',
 'the',
 'time',
 'in',
 'every',
 'moment',
 'of',
 'existence.',
 'Try',
 'to',
 'figure',
 'the',
 'hard',
 'times',
 'as',
 'times',
 'of',
 'growth,',
 'as...',
 '84389',
 '84390',
 '...',
 'Welcome',
 'and',
 'stuff.',
 'Game.',
 'Set.',
 'Match.',
 'Prozac,',
 'wellbrutin,',
 'at',
 'least',
 'thirty',
 'minu

In [43]:
fruits = {"apple", "banana", "cherry"}

fruits.discard("banana")

print(fruits)

{'cherry', 'apple'}


In [61]:
df_raw_no_quote.shape[0]

650

In [ ]:
import re
re.split(' |\|\|\|',str)

In [ ]:
df_raw_no_quote.posts = df_raw_no_quote.posts.str.split(' ')
df_raw_no_quote.head()